In [ ]:
import cv2
import os
import numpy as np
import gc

folder_path = "F:/CyberAI/"

input_path = folder_path + "raw_data/"
output_path = folder_path + "small_data/"

i = 0
first_file = 0
last_file = 11204

report_steps = 1000
gc_steps = 1000

new_size = (256, 256)

for filename in os.listdir(input_path)[first_file:last_file]:
    i += 1
    if i % report_steps == 0: 
        print(i)
        if i % gc_steps == 0:
            gc.collect()
        
    img = cv2.imread(os.path.join(input_path, filename))
    img = cv2.resize(img, new_size)
    cv2.imwrite(output_path + filename, img) 

In [ ]:
folder_path = "F:/CyberAI/"

In [ ]:
import gc
gc.collect()

In [ ]:
# size of image array

x1 = 256
y1 = 256

In [ ]:
import cv2
import os
import numpy as np

data_path = folder_path + "small_data/"


images_normal = []
images_mirrored = []

wheel_normal = []
wheel_mirrored = []

# mirror data set to double it and remove directional bias (e.g. always turning left on a round track)

i = 0
for filename in os.listdir(data_path):#[10_000:30_000]:
    i += 1
    if i % report_steps == 0: 
        print(i)
    if i % 1 == 0: 
        img = cv2.imread(os.path.join(data_path,filename))
        img = np.array(img,dtype=np.uint8)
        # uint can also be float32, float16 etc
        if img is not None:
            # parse img and filename into data
            data = np.array(filename.replace(".jpg","").split("_")[-3:],dtype=np.float32)
            # if abs(data[0]) < 0.5:  -  if betrag von data kleiner als (extreme auslassen)
            
            images_normal.append(img/1)
            wheel_normal.append(float(data[0]))

            images_mirrored.append(cv2.flip(img, 1))
            wheel_mirrored.append(float(data[0]*-1))
            # "cv2.flip" -> mirroring, wheel * -1
            
# print(wheel_normal)

In [ ]:
image_set = []
wheel_set = []

for i in range(len(wheel_normal)-1):
    try:
        wheel_set.append((0.5*wheel_normal[i-1]+wheel_normal[i]+0.5*wheel_normal[i+1])/2)
    except: 
        print("fuck")
for i in range(len(wheel_mirrored)-1):
    try:
        wheel_set.append((0.5*wheel_mirrored[i-1]+wheel_mirrored[i]+0.5*wheel_mirrored[i+1])/2)    
    except: 
        print("fuck")
        
image_set = images_normal[1:] + images_mirrored[1:]

In [ ]:
import random
import gc

shuffled_set = list(zip(image_set, wheel_set))
random.shuffle(shuffled_set)
image_set, wheel_set = zip(*shuffled_set)

gc.collect()

# print(str(image_set[0]) + ", " + str(wheel_set[0]))

In [ ]:
import numpy as np
import copy
import gc

# macht array draus. Weil Tensorflow komishc ist und keine Listen kann
wheel_set = np.array(wheel_set).reshape(-1,1)
image_set = np.array(image_set,dtype=np.float32)

print("FORMATING DONE")

In [ ]:
print(np.min(wheel_set))
print(np.max(wheel_set))
print(np.min(image_set))
print(np.max(image_set))

In [ ]:
gc.collect()
print(wheel_set.shape)
print(image_set.shape)
gc.collect()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Input, Dense, Flatten,Concatenate,UpSampling2D, BatchNormalization, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D,Reshape, Reshape, Permute
from tensorflow.keras.optimizers import Adam,Adadelta 
from tensorflow.keras.metrics import MeanSquaredError, Accuracy
from tensorflow.keras.losses import huber
from tensorflow.keras.models import Model

In [ ]:
input1 = keras.layers.Input(shape=(y1,x1,3))

Y = Conv2D(16, (3,3),padding = 'valid')(input1)
Y = MaxPooling2D(pool_size=(2, 2), padding='valid')(Y)

Y = Conv2D(32, (3,3),padding = 'valid')(Y)
Y = MaxPooling2D(pool_size=(2, 2), padding='valid')(Y)

Y = Conv2D(64, (3,3),padding = 'valid')(Y)
Y = MaxPooling2D(pool_size=(2, 2), padding='valid')(Y)

Y = Conv2D(128, (3,3),padding = 'valid')(Y)

Y = Conv2D(1, (3,3),padding = 'valid')(Y)

Y = Flatten()(Y)

Z = Dense(1)(Y)


model = keras.models.Model(input1,Z)
#X = Dense(16)(X)
#Y = Conv2D(32, (5,5),padding = 'valid')(Y)
#Y = MaxPooling2D(pool_size=(2, 2), padding='valid')(Y)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0002), loss='mean_squared_error', metrics=['mean_absolute_error', 'mean_squared_error'])

In [ ]:
model.fit(image_set, wheel_set, batch_size=1, epochs=10, validation_split=0.10, verbose=1)#,callbacks=[tensorboard])

In [ ]:
model_path = folder_path + "AI1.model"

model.save(model_path)

In [ ]:
# start here if youre just using the model

model_path = folder_path + "AI1.model"

model = tf.keras.models.load_model(model_path)

In [ ]:
def axiconvertion(angle):
    angle = (((angle)+1)/2)*33280
    #median = int(33280//2 + 5000)
    #angle  = 1 * angle
    #if angle >= 0:
    #    Axis = median + 3*angle* (33280-median)
    #else:
    #    Axis = median + angle*median
    
    return int(angle)

In [ ]:
import pyvjoy

#Pythonic API, item-at-a-time

j = pyvjoy.VJoyDevice(1)

#turn button number 15 on
j.set_button(2,1)

#Notice the args are (buttonID,state) whereas vJoy's native API is the other way around.


#turn button 15 off again
j.set_button(2,0)

#Set X axis to fully left
j.set_axis(pyvjoy.HID_USAGE_X, 0x1)

#Set X axis to fully right
j.set_axis(pyvjoy.HID_USAGE_X, 0x8000)

#Also implemented:

j.reset()
j.reset_buttons()
j.reset_povs()



#The 'efficient' method as described in vJoy's docs - set multiple values at once

j.data



j.data.lButtons = 19 # buttons number 1,2 and 5 (1+2+16)
j.data.wAxisX = 0x200 
j.data.wAxisY= 0x3a15

#send data to vJoy device
j.update()


#Lower-level API just wraps the functions in the DLL as thinly as possible, with some attempt to raise exceptions instead of return codes.

In [ ]:
import time
from mss import mss
import cv2
import numpy as np
import copy


cords = {'top':32-1080 , 'left': 0 , 'width':1280  , 'height':1024}
sct = mss()
frames = []
smoothwheel = 0

    
while True:

    frame = (np.array(sct.grab(cords))[:,:,:3])
    frame = cv2.resize(frame, (x1,y1))
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    frame = cv2.resize(frame, (x1,y1)).reshape(1,y1,x1,3)
    frame = np.array(frame)
    


    prediction = np.average(model.predict(frame)*1)
    smoothwheel = 0.3 * prediction + 0.7*smoothwheel
    
    wheel = axiconvertion(prediction)
    j.set_axis(pyvjoy.HID_USAGE_X, int(wheel))
    j.set_axis(pyvjoy.HID_USAGE_Z, int(4000))
    

    showframe = copy.deepcopy(frame).reshape(y1,x1,3)
    showframe = cv2.line(showframe, (x1//2, y1),(x1//2,0), (255, 255, 255), thickness=1)
    showframe = cv2.line(showframe, (x1//2, y1), (x1//2+int(smoothwheel*200), int(50+(smoothwheel**2)*100)), (255, 0, 255), thickness=1)
    showframe = cv2.line(showframe, (x1//2, y1), (x1//2+int(prediction*200), int(50+(prediction**2)*100)), (255, 255, 0), thickness=2)
    showframe = cv2.resize(showframe, (3*x1,3*y1))
    showframe = cv2.putText(np.array(showframe,dtype=np.uint8), str(prediction), (40, 40), cv2.FONT_HERSHEY_SIMPLEX , 1, (255,0,255), 3)  
    
    cv2.imshow("showframe", showframe)  
    #cv2.imshow("new_img", new_img)  


    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [ ]:
import time

while True:
    #j.set_axis(pyvjoy.HID_USAGE_Y, int(33280//2 + 5000))
    j.set_axis(pyvjoy.HID_USAGE_Y, 0)
    time.sleep(1)
    j.set_axis(pyvjoy.HID_USAGE_Y, 33280)
    time.sleep(1)
    #j.update()

#send data to vJoy device
j.update()